This notebook presents the appplication of Bayesian Optimisation to the problem of optimally controlling the 
stretcher parameters of the L1 pump Laser.

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague

# Setting
The goal of this project is to maximise second-harmonic efficiency. 

However, since this metric is also very much related to the shortest possible pulse shape, we developed a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which is, by default, the shortest one).

This process is very much intensive for what concerns the number of function evaluation we needed to use and, because of that, we resorted to use a custom-built fast-running forward model used to approximate/predict the temporal  profile of the pulse given a set of parameters. 

In particular, if one indicates with $\psi = \big( d_1, d_2, d_3 \big)$ the control configuration of the stretcher and with $E(\nu)$ the electric field in the frequency domain of the signal considered, then the fast running model we developed approximates $ \tilde{\tau}(\psi; E(\nu)) $, hopefully similar enough to the actual ${\tau}(\psi; E(\nu))$ that can be collected only through practical observation on the real hardware. 

If one has a target temporal profile $\tau^*$ of the pulse then it the problem can be framed as: 

$$\min_{\psi} L\big[ \tilde{\tau}(\psi; E(\nu)), \tau^*\big]$$

With $L$ being some sort of loss function. 

The solution of this problem can be obtained, in principle, using Bayesian Optimisation. In this notebook, this is the approach that is tried out. 

Clearly enough, the solution to this problem  depends on the Loss function being used. 

In [1]:
# these imports are necessary to import modules from directories one level back in the folder structure
import sys
sys.path.append("..")

from algorithms.L1_BayesianOptimisation import *
from utils import physics

In [2]:
frequency, intensity = extract_data() # extracting the desired information

# compressor parameters - obtained as minus the stretcher one - TO BE FIXED WITH REAL ONES ONCE THEY ARE AVAILABLE
COMPRESSOR = -1 * np.array((267.422 * 1e-24, -2.384 * 1e-36, 9.54893 * 1e-50)) # in s^2, s^3 and s^4 (SI units)
# non linearity parameter
B = 2
# cutoff frequencies, in THz, used to remove noise - derived from visual inspection
CUTOFF = (289.95, 291.91)
# model instantiation
l1_pump = model(frequency, intensity, COMPRESSOR, B, CUTOFF, num_points=int(5e3))
# pre-processed version of frequency and intensity
frequency_clean, intensity_clean = l1_pump.spit_center()

# target temporal profile shape
_, target_profile = physics.temporal_profile(frequency_clean, np.sqrt(intensity_clean),
                                             phase = np.zeros_like(frequency_clean),
                                             npoints_pad = l1_pump.pad_points)

temporal_profile = lambda d2, d3, d4: l1_pump.forward_pass(np.array((d2, d3, d4)))[1]
objective_function = lambda d2, d3, d4: -1 * physics.mse(temporal_profile(d2, d3, d4), target_profile)

/Users/francescocapuano/Dropbox (Politecnico Di Torino Studenti)/Mac/Desktop/ELI/ELIopt/data/L1_pump_spectrum.csv


The bounds of the optimisation process are defined considering that: 

1. The value of GDD is tunable only in a region which is a little bit more than $\pm$ 10% of its nominal value.

2. TOD is tunable practically without limits in the case in which the bounds are defined using percentages of the nominal values. 

3. FOD is tunable practically without limits in the case in which the bounds are defined using percentages of the nomnal values. 

In [3]:
GDDperc, TODperc, FODperc = 0.1, 0.2, 0.3
# stretcher control bounds are centered in the compressor bounds and have a width related to a given percentage
# (which can be though of as an hyperparameter as long as it is in the tunable interval)

low_stretcher, high_stretcher = (-1 * COMPRESSOR * np.array((1 - GDDperc, 1 - TODperc, 1 - FODperc)), 
                                 -1 * COMPRESSOR * np.array((1 + GDDperc, 1 + TODperc, 1 + FODperc))
                                )

# stretcher control must be given in terms of dispersion coefficients so they must be translated into d2, d3 and d4. 
low_stretcher, high_stretcher = (l1_pump.translate_control(low_stretcher, verse = "to_disp"),
                                 l1_pump.translate_control(high_stretcher, verse = "to_disp")
                                )

# these are the bounds for the parameter currently optimized - sign can change so sorting is used
pbounds = {
    "d2": np.sort((low_stretcher[0], high_stretcher[0])), 
    "d3": np.sort((low_stretcher[1], high_stretcher[1])), 
    "d4": np.sort((low_stretcher[2], high_stretcher[2]))
}

In [4]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
n_init, n_iter = 100, 400

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

|   iter    |  target   |    d2     |    d3     |    d4     |
-------------------------------------------------------------
|  2        | -0.008716 | -0.4505   | -6.578e+0 | -5.744e+1 |
|  3        | -0.004994 | -0.5027   | -5.841e+0 | -5.912e+1 |
|  6        | -0.004145 | -0.4634   | -5.95e+06 | -5.541e+1 |
|  9        | -0.000605 | -0.4796   | -6.759e+0 | -4.556e+1 |
|  16       | -0.000276 | -0.4746   | -7.896e+0 | -4.477e+1 |


In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

Typically Bayesian Optimisation is really sample-efficient for what concerns the number of function evaluations so 350 iterations should really be enough. 

The result of such iterations are presented in the the following Figure. 

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)

In [ ]:
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)
ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

The above Figure shows: 
1. *Target Pulse*: defined as the Transform Limited pulse (the one having $\varphi = 0$) of the given spectrum. 

2. *The temporal profile of the controlled pulse when the phase of the stretcher is equal and opposite to the one of the compressor* (this would be the transform limited pulse if no non-linear effect were present in the system)

3. *The Bayesian Optimisation output*, which is very different from the desired shape even if it certainly does approach a shape way more quasi-Gaussian than the ideal linear control. 

This however is valid for an optimization process that would use as loss function $\frac 1N \Vert t - t_{TL} \Vert^2$, with $N$ being the number of points in the array (i.e., using the MSE error function). 
Surely enough other loss functions can be defined. In particular: 

1. One which **weights every error by the value of the intensity**, so as to reduce the weight of the marginal errors with respect to those large ones present in the central area of the pulse, i.e. $$ \frac{\Vert t (t - t_{TL})\Vert^2}{t^T \mathbb 1}$$

2. One which computes the **MSE only for non-zero values**, and which therefore discards those long tails of the pulse.

3. A mixture of (2) and a measure of **the mismatch in terms of area underlied by controlled and target pulse** (weighted in favour of difference of area).

4. One which mixes (2) with a minimisation of the total sum of peak widhts (weighted in favour of difference of total width sum).

5. One which minimises the pulse shape but prefers to **minimize the difference in FWHM** between the target pulse and the controlled one. 

6. One which is based on *L1-Manhattan norm*

In [ ]:
mse_pulse = profile_BO

In [ ]:
# loss-1
def objective_function(d2, d3, d4): 
    controlled_profile = temporal_profile(d2, d3, d4)
    return -1 * (controlled_profile * (controlled_profile - target_profile)**2).sum()/controlled_profile.sum()

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)
ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss1_pulse = profile_BO

In [ ]:
# loss-2
def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    controlled_profile = temporal_profile(d2, d3, d4)
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    
    return -1 * ((controlled_profile - target)**2).mean()

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)
ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss2_pulse = profile_BO

In [ ]:
# loss-3
def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    w1, w2 = 0.3, 0.7
    controlled_profile = temporal_profile(d2, d3, d4)
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    
    return -1 * (w1 * ((controlled_profile - target)**2).mean() + w2 * (np.trapz(controlled_profile) - np.trapz(target)))

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)

ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss3_pulse = profile_BO

In [ ]:
from scipy.signal import find_peaks, peak_widths
# loss-4
def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    w1, w2 = 0.3, 0.7
    controlled_profile = temporal_profile(d2, d3, d4)
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    
    control_peaks, _ = find_peaks(controlled_profile, height = (0.1, None))
    target_peaks, _ = find_peaks(target, height = (0.1, None))
    
    control_width = peak_widths(controlled_profile, control_peaks)[0]
    target_width = peak_widths(target, target_peaks)[0]
    
    mse_part = ((controlled_profile - target)**2).mean() 
    peak_part = (target_width.sum() - control_width.sum())**2
    return -1 * (w1 * mse_part + w2 * peak_part)

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)

ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss4_pulse = profile_BO

In [ ]:
# loss-5
from scipy.interpolate import UnivariateSpline

target_time, target_profile = physics.temporal_profile(frequency_clean, np.sqrt(intensity_clean),
                                             phase = np.zeros_like(frequency_clean),
                                             npoints_pad = l1_pump.pad_points)

def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    w1, w2 = 0.1, 0.9
    controlled_time, controlled_profile = l1_pump.forward_pass(np.array((d2, d3, d4)))
    
    control_roots = UnivariateSpline(
        x = controlled_time, y = controlled_profile - (controlled_profile.max()/2), s = 0
    ).roots()
        
    control_FWHM = np.abs(control_roots[0] - control_roots[-1])
    target_FWHM = np.diff(UnivariateSpline(
        x = target_time, y = target_profile - (target_profile.max()/2), s = 0
    ).roots())
    
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    
    shape_part = ((controlled_profile - target)**2).mean()
    FWHM_part = np.abs((control_FWHM - target_FWHM).item())
    
    return -1 * (w1 * shape_part + w2 * FWHM_part)

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)

ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss5_pulse = profile_BO

In [ ]:
#loss-6
def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    w1, w2 = 0.1, 0.9
    
    controlled_profile = temporal_profile(d2, d3, d4)
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    

    return -1 * (np.abs(controlled_profile - target)).sum()

In [ ]:
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=10, # for reproducibility
)

# these are the actual hyperparameters of the optimization process
# n_init, n_iter = 150, 200

optimizer.maximize(
    init_points=n_init,
    n_iter=n_iter,
)

In [ ]:
# plotting the result obtained with Bayesian Optimization
optimald2, optimald3, optimald4 = optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]

optimal_control = (optimald2, optimald3, optimald4)
# applying optimal control found
time_BO, profile_BO = l1_pump.forward_pass(optimal_control)

print("Final MSE between Control and Target: {:.4e}".format(-1*objective_function(optimald2, optimald3, optimald4)))

In [ ]:
# output if non-linearity was absent from the system 
linear_opt_time, linear_opt_profile = l1_pump.forward_pass(
    control = l1_pump.translate_control(
        -1*COMPRESSOR, verse = "to_disp"
    )
)
control = np.array((optimizer.max["params"]["d2"], optimizer.max["params"]["d3"], optimizer.max["params"]["d4"]))
time_BO, profile_BO = l1_pump.forward_pass(control)
profile_BO = np.roll(profile_BO, np.argmax(target_profile) - np.argmax(profile_BO))

fig, ax = plt.subplots(figsize = (10, 8))
ax.set_title("Pulse Optimization results", fontsize = 12, fontweight = "bold")
ax.plot(time_BO, profile_BO, label = "Bayesian Optimization output", lw = 4)
ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)

ax.scatter(linear_opt_time, np.roll(linear_opt_profile, 
                                    - linear_opt_profile.argmax() + target_profile.argmax()),
           label = r"$\varphi_{stretcher} = - \varphi_{compressor}$", marker = "o", s = 25, c = "k")

ax.set_xlabel(r"Time (s)", fontsize = 12); ax.set_ylabel("Intensity", fontsize = 12)

ax.legend(fontsize = 13)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
fig.tight_layout()
plt.show()

In [ ]:
loss6_pulse = profile_BO

The different loss functions have given different results.

In [ ]:
# final comparison
fig, ax = plt.subplots(figsize = (10,8))
ax.plot(time_BO, np.roll(mse_pulse, - mse_pulse.argmax() + target_profile.argmax()), label = "Naive MSE")

ax.plot(time_BO, np.roll(loss1_pulse, - loss1_pulse.argmax() + target_profile.argmax()), label = "Loss1", lw = 3)
ax.plot(time_BO, np.roll(loss2_pulse, - loss2_pulse.argmax() + target_profile.argmax()), label = "Loss2", lw = 3)
ax.plot(time_BO, np.roll(loss3_pulse, - loss3_pulse.argmax() + target_profile.argmax()), label = "Loss3", lw = 3)
ax.plot(time_BO, np.roll(loss4_pulse, - loss4_pulse.argmax() + target_profile.argmax()), label = "Loss4", lw = 3)
ax.plot(time_BO, np.roll(loss5_pulse, - loss5_pulse.argmax() + target_profile.argmax()), label = "Loss5", lw = 3)
ax.plot(time_BO, np.roll(loss6_pulse, - loss6_pulse.argmax() + target_profile.argmax()), label = "Loss6", lw = 3)

ax.scatter(time_BO, target_profile, label = "Target Pulse", c = "tab:grey", marker = "x", s = 50)

ax.set_xlabel("Time (s)", fontsize = 13); ax.set_ylabel("Intensity", fontsize = 13)
ax.set_title("Optimal pulse shape obtained with various loss functions", fontweight = "bold", fontsize = 14)

ax.legend(fontsize = 12)
ax.set_xlim(left = -0.75e-11, right = +0.75e-11)
plt.show()

None of the objective functions which have been tried out has shown to be good enough to stimulate convergence, so it is necessary to try out other approaches or (far more important) go on with a phase of hyper-parameter tuning for the optimizer once a promising objective function has been developed. 

The sixth loss function (L1-Manhattan Norm) has been proven to be the best one so far, so hyperparameter tuning will be studied on this very loss function. 

However, these experiments should be interpreted by the standpoint of the actual difficulty of defyning parts so relevant of the problem (such as the objective function, indeed). This is something that makes me doubt about the effectviness of these approaches (whose outcome heavily relies on human choices which are very likely to be suboptimal), and makes me think that, maybe, approaches based on *Reinforcement Learning* would serve the scope better.